In [21]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
path = os.path.join(os.getcwd(), os.pardir)
sys.path.append(path)

In [22]:
import pandas as pd
import numpy as np
from scipy.stats import norm, skew
from sklearn import metrics
import matplotlib.pyplot as plt
import pylab as py
from BAI import environment, solver, analysis

In [23]:
plt.rcParams['figure.figsize'] = [10, 6]

# Two Batch Simulation

In [24]:
K = 3
n_parallels = 100000
n_batches = 2
batch_func = environment.func_poisson_batch(1000)
best_mu = 0.01
reward_func = environment.func_gaussian_reward([best_mu, 0.0, 0.0], [1, 1, 1])
bandit = environment.Bandit(K=K, reward_func=reward_func, batch_func=batch_func)

# WB-TS-Phi-A

In [25]:
G = 5000
tracking_history = []
for parallel_idx in range(n_parallels):
    SS_phi = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    optimal_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        sampling_probs = [optimal_probs[i] / np.sum(optimal_probs) for i in range(K)]
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        batch_size = batch_feedback['batch_size']
        for arm_idx in range(K):
            batch_rewards = batch_feedback['rewards'][arm_idx]
            batch_trials = batch_feedback['trials'][arm_idx]
            phi = 1
            SS_phi[arm_idx] += phi ** 2
            numerator[arm_idx] += phi * np.sqrt(batch_trials) * (batch_rewards / batch_trials)
            denominator[arm_idx] += phi * np.sqrt(batch_trials)
        mc_sampling = np.array([np.random.normal(numerator[i]/denominator[i], np.sqrt(1 * SS_phi[i]/denominator[i]**2), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                numerator[arm_idx]/denominator[arm_idx],
                denominator[arm_idx]**2 / SS_phi[arm_idx],
                optimal_probs[arm_idx],
                posterior_delta[arm_idx]
            )
            tracking_history.append(tracking_reccord)
# convert tracking_history to dataframe
df_WBTS_A = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu', 'tau', 'optimal_prob', 'posterior_delta'
])

# WB-TS-Phi-B

In [26]:
G = 5000
tracking_history = []
for parallel_idx in range(n_parallels):
    SS_phi = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    optimal_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        sampling_probs = [optimal_probs[i] / np.sum(optimal_probs) for i in range(K)]
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        batch_size = batch_feedback['batch_size']
        for arm_idx in range(K):
            batch_rewards = batch_feedback['rewards'][arm_idx]
            batch_trials = batch_feedback['trials'][arm_idx]
            phi = np.sqrt(batch_size)
            SS_phi[arm_idx] += phi ** 2
            numerator[arm_idx] += phi * np.sqrt(batch_trials) * (batch_rewards / batch_trials)
            denominator[arm_idx] += phi * np.sqrt(batch_trials)
        mc_sampling = np.array([np.random.normal(numerator[i]/denominator[i], np.sqrt(1 * SS_phi[i]/denominator[i]**2), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                numerator[arm_idx]/denominator[arm_idx],
                denominator[arm_idx]**2 / SS_phi[arm_idx],
                optimal_probs[arm_idx],
                posterior_delta[arm_idx]
            )
            tracking_history.append(tracking_reccord)
# convert tracking_history to dataframe
df_WBTS_B = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu', 'tau', 'optimal_prob', 'posterior_delta'
])

In [27]:
mus_A = df_WBTS_A.query("batch_idx == 1 and arm_idx == 0").mu - best_mu
taus_A = df_WBTS_A.query("batch_idx == 1 and arm_idx == 0").tau
mus_B = df_WBTS_B.query("batch_idx == 1 and arm_idx == 0").mu - best_mu
taus_B = df_WBTS_B.query("batch_idx == 1 and arm_idx == 0").tau

print(
    np.mean(1/np.sqrt(taus_A)),
    np.std(1/np.sqrt(taus_A)), 
    np.mean(1/np.sqrt(taus_B)),
    np.std(1/np.sqrt(taus_B))
)


0.0415025317973709 0.010085139139375312 0.04146883405815654 0.010125570822208472


In [28]:
mus_A = df_WBTS_A.query("batch_idx == 1 and arm_idx == 1").mu
taus_A = df_WBTS_A.query("batch_idx == 1 and arm_idx == 1").tau
mus_B = df_WBTS_B.query("batch_idx == 1 and arm_idx == 1").mu
taus_B = df_WBTS_B.query("batch_idx == 1 and arm_idx == 1").tau

print(
    np.mean(1/np.sqrt(taus_A)),
    np.std(1/np.sqrt(taus_A)), 
    np.mean(1/np.sqrt(taus_B)),
    np.std(1/np.sqrt(taus_B))
)

0.04371112358373282 0.01083431008014122 0.043767303687975434 0.010845785456463274


# WB-TTTS-Phi-A

In [29]:
G = 5000
tracking_history = []
for parallel_idx in range(n_parallels):
    SS_phi = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    optimal_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        optimal_odds = [optimal_probs[i] / (1 - optimal_probs[i]) if optimal_probs[i] < 1 else 99 for i in range(K)]
        sampling_probs = [optimal_probs[i] * 0.5 * (1 + np.sum(optimal_odds) - optimal_odds[i]) for i in range(K)]
        sampling_probs = [sampling_probs[i] / np.sum(sampling_probs) for i in range(K)]
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        batch_size = batch_feedback['batch_size']
        for arm_idx in range(K):
            batch_rewards = batch_feedback['rewards'][arm_idx]
            batch_trials = batch_feedback['trials'][arm_idx]
            phi = np.sqrt(batch_size)
            SS_phi[arm_idx] += phi ** 2
            numerator[arm_idx] += phi * np.sqrt(batch_trials) * (batch_rewards / batch_trials)
            denominator[arm_idx] += phi * np.sqrt(batch_trials)
        mc_sampling = np.array([np.random.normal(numerator[i]/denominator[i], np.sqrt(1 * SS_phi[i]/denominator[i]**2), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                numerator[arm_idx]/denominator[arm_idx],
                denominator[arm_idx]**2 / SS_phi[arm_idx],
                optimal_probs[arm_idx],
                posterior_delta[arm_idx]
            )
            tracking_history.append(tracking_reccord)
# convert tracking_history to dataframe
df_WBTTTS_A = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu', 'tau', 'optimal_prob', 'posterior_delta'
])

# WB-TTTS-Phi-B

In [30]:
G = 5000
tracking_history = []
for parallel_idx in range(n_parallels):
    SS_phi = [0 for i in range(K)]
    numerator = [0 for i in range(K)]
    denominator = [0 for i in range(K)]
    optimal_probs = [1 / K for i in range(K)]
    for batch_idx in range(n_batches):
        optimal_odds = [optimal_probs[i] / (1 - optimal_probs[i]) if optimal_probs[i] < 1 else 99 for i in range(K)]
        sampling_probs = [optimal_probs[i] * 0.5 * (1 + np.sum(optimal_odds) - optimal_odds[i]) for i in range(K)]
        sampling_probs = [sampling_probs[i] / np.sum(sampling_probs) for i in range(K)]
        batch_feedback = bandit.batch_observe_fast(
            sampling_distrib = sampling_probs, 
            time_index=batch_idx
        )
        batch_size = batch_feedback['batch_size']
        for arm_idx in range(K):
            batch_rewards = batch_feedback['rewards'][arm_idx]
            batch_trials = batch_feedback['trials'][arm_idx]
            phi = np.sqrt(batch_size)
            SS_phi[arm_idx] += phi ** 2
            numerator[arm_idx] += phi * np.sqrt(batch_trials) * (batch_rewards / batch_trials)
            denominator[arm_idx] += phi * np.sqrt(batch_trials)
        mc_sampling = np.array([np.random.normal(numerator[i]/denominator[i], np.sqrt(1 * SS_phi[i]/denominator[i]**2), G) for i in range(K)])
        mc_optimal_arm = np.argmax(mc_sampling, axis=0)
        posterior_delta = np.mean(np.max(mc_sampling, axis=0) - mc_sampling, axis=1)
        optimal_probs = [round(np.sum(np.where(mc_optimal_arm == arm_i, 1, 0)) / G, 4) for arm_i in range(K)]
        # tracking history and analysis
        for arm_idx in range(K):
            tracking_reccord = (
                parallel_idx,
                batch_idx,
                arm_idx,
                sampling_probs[arm_idx],
                numerator[arm_idx]/denominator[arm_idx],
                denominator[arm_idx]**2 / SS_phi[arm_idx],
                optimal_probs[arm_idx],
                posterior_delta[arm_idx]
            )
            tracking_history.append(tracking_reccord)
# convert tracking_history to dataframe
df_WBTTTS_B = pd.DataFrame(tracking_history, columns=[
    'parallel_idx','batch_idx', 'arm_idx', 'sampling_prob', 
    'mu', 'tau', 'optimal_prob', 'posterior_delta'
])

In [31]:
mus_A = df_WBTTTS_A.query("batch_idx == 1 and arm_idx == 0").mu - best_mu
taus_A = df_WBTTTS_A.query("batch_idx == 1 and arm_idx == 0").tau
mus_B = df_WBTTTS_B.query("batch_idx == 1 and arm_idx == 0").mu - best_mu
taus_B = df_WBTTTS_B.query("batch_idx == 1 and arm_idx == 0").tau

print(
    np.mean(1/np.sqrt(taus_A)),
    np.std(1/np.sqrt(taus_A)), 
    np.mean(1/np.sqrt(taus_B)),
    np.std(1/np.sqrt(taus_B))
)


0.03969392775674247 0.006744962192385204 0.03969416430693356 0.00672812911548662


In [32]:
mus_A = df_WBTTTS_A.query("batch_idx == 1 and arm_idx == 1").mu - best_mu
taus_A = df_WBTTTS_A.query("batch_idx == 1 and arm_idx == 1").tau
mus_B = df_WBTTTS_B.query("batch_idx == 1 and arm_idx == 1").mu - best_mu
taus_B = df_WBTTTS_B.query("batch_idx == 1 and arm_idx == 1").tau

print(
    np.mean(1/np.sqrt(taus_A)),
    np.std(1/np.sqrt(taus_A)), 
    np.mean(1/np.sqrt(taus_B)),
    np.std(1/np.sqrt(taus_B))
)


0.04102840727213576 0.007773828951716675 0.04098685157801637 0.007747217521924362
